In [40]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator

from indra_db_lite.api import get_entrez_pmids_for_hgnc
from indra_db_lite.api import get_entrez_pmids_for_uniprot
from indra_db_lite.api import get_plaintexts_for_text_ref_ids
from indra_db_lite.api import get_text_ref_ids_for_agent_text
from indra_db_lite.api import get_text_ref_ids_for_pmids


from adeft_indra.grounding import AdeftGrounder
from adeft_indra.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename

In [41]:
def get_text_ref_ids_for_entity(ns, id_):
    if ns == 'HGNC':
        pmids = get_entrez_pmids_for_hgnc(id_)
    elif ns == 'UP':
        pmids = get_entrez_pmids_for_uniprot(id_)
    return list(get_text_ref_ids_for_pmids(pmids).values())

In [42]:
adeft_grounder = AdeftGrounder()

In [43]:
shortforms = ['SPO']
model_name = '&'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../../', 'results', model_name))

In [44]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    text_ref_ids = get_text_ref_ids_for_agent_text(shortform)
    content = get_plaintexts_for_text_ref_ids(text_ref_ids, contains=shortforms)
    text_dict = content.flatten()
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [48]:
z = [x for x in p if 'sporulation' in x]

In [54]:
p

dict_values(['In mammals, peroxidases have some organ, tissue, cellular and sub-cellular specific distribution patterns, performing some specific functions. These peroxidases include, glutathione peroxidase (GPx), myeloperoxidase (MPO), eosinophil peroxidase (EPO), uterine peroxidase, lactoperoxidase (LPO), salivary peroxidase (SPO) and thyroid peroxidase (TPO) (Klebanoff, 2005). These peroxidases play an important role in wide metabolic activities and are directly or indirectly involved in various diseases (Cheng et al., 2008).\nDifferent harmful microorganisms enter the human body through the oral passage regularly. These bacteria are killed by the first line of defence system present in saliva, which includes salivary peroxidase (SPO) as the major peroxidase. Oral peroxidases OPO are composed of salivary peroxidase (80%) and MPO (20%) (Pruitt et al., 1990; Nagler et al., 2002). SPO inhibits both Gram-positive and Gram-negative oral and non-oral bacteria. In addition to this, SPO sho

In [55]:
list(zip(longforms, counts))

[('sporulation medium', 11),
 ('salivary peroxidase', 3),
 ('sodium percarbonate', 3)]

In [56]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

KeyboardInterrupt: 

In [ ]:
result = [grounding_map, names, pos_labels]

In [33]:
result

[{'salivary peroxidase': 'HGNC:6678',
  'sodium percarbonate': 'MESH:C034738',
  'sporulation medium': 'EFO:EFO:0007033'},
 {'HGNC:6678': 'LPO',
  'MESH:C034738': 'sodium percarbonate',
  'EFO:EFO:0007033': 'SPO'},
 ['EFO:EFO:0007033', 'HGNC:6678']]

In [57]:
grounding_map, names, pos_labels = [{'salivary peroxidase': 'HGNC:6678',
  'sodium percarbonate': 'MESH:C034738',
  'sporulation medium': 'EFO:EFO:0007033'},
 {'HGNC:6678': 'LPO',
  'MESH:C034738': 'sodium percarbonate',
  'EFO:EFO:0007033': 'SPO'},
 ['EFO:EFO:0007033', 'HGNC:6678']]

In [58]:
excluded_longforms = []

In [59]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [67]:
additional_entities = {
   'HGNC:6678': ['LPO', ['LPO', 'SPO', 'lactoperoxidase', 'peroxidase']],
   'HGNC:16507': ['SYNPR', ['SPO', 'SYNPR', 'synaptoporin', 'MGC26651']],
}

In [69]:
unambiguous_agent_texts = {
    'NCIT:NCIT:C60832': ['Oxygen Saturation Measurement', ['oxygen saturation']],
}

In [70]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts(
    (text, text_ref_id) for text_ref_id, text in all_texts.items()
)
agent_text_text_ref_id_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_text_ref_id_map[label].append(id_)

entity_text_ref_id_map = {
    entity: set(
        get_text_ref_ids_for_entity(*entity.split(':', maxsplit=1))
    )
    for entity in additional_entities
}

In [71]:
t = [x for x in corpus if x[1].startswith('EFO')]

In [73]:
len(corpus)

6

In [74]:
intersection1 = []
for entity1, trids1 in entity_text_ref_id_map.items():
    for entity2, trids2 in entity_text_ref_id_map.items():
        intersection1.append((entity1, entity2, len(trids1 & trids2)))

In [75]:
intersection2 = []
for entity1, trids1 in agent_text_text_ref_id_map.items():
    for entity2, pmids2 in entity_text_ref_id_map.items():
        intersection2.append((entity1, entity2, len(set(trids1) & trids2)))

In [76]:
intersection1

[('HGNC:6678', 'HGNC:6678', 33),
 ('HGNC:6678', 'HGNC:16507', 5),
 ('HGNC:16507', 'HGNC:6678', 5),
 ('HGNC:16507', 'HGNC:16507', 20)]

In [77]:
intersection2

[('HGNC:6678', 'HGNC:6678', 0),
 ('HGNC:6678', 'HGNC:16507', 0),
 ('EFO:EFO:0007033', 'HGNC:6678', 0),
 ('EFO:EFO:0007033', 'HGNC:16507', 0),
 ('MESH:C034738', 'HGNC:6678', 0),
 ('MESH:C034738', 'HGNC:16507', 0)]

In [78]:
all_used_trids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_trids = set()
    for agent_text in agent_texts[1]:
        trids = set(get_text_ref_ids_for_agent_text(agent_text))
        new_trids = list(trids - all_texts.keys() - used_trids)
        content = get_plaintexts_for_text_ref_ids(new_trids, contains=agent_texts[1])
        text_dict = content.flatten()
        corpus.extend(
            [
                (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
            ]
        )
        used_trids.update(new_trids)
    all_used_trids.update(used_trids)
        
for entity, trids in entity_text_ref_id_map.items():
    new_trids = list(set(trids) - all_texts.keys() - all_used_trids)
    _, contains = additional_entities[entity]
    content = get_plaintexts_for_text_ref_ids(new_trids, contains=contains)
    text_dict = content.flatten()
    corpus.extend(
        [
            (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
        ]
    )

In [79]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [80]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2021-10-12 20:44:38] /adeft/Py/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2021-10-12 20:44:42] /adeft/Py/adeft/adeft/modeling/classify.py - Best f1 score of 0.9698476343223736 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [81]:
classifier.stats

{'label_distribution': {'HGNC:6678': 22,
  'EFO:EFO:0007033': 3,
  'MESH:C034738': 1,
  'NCIT:NCIT:C60832': 186,
  'HGNC:16507': 4},
 'f1': {'mean': 0.969848, 'std': 0.009264},
 'precision': {'mean': 0.967653, 'std': 0.011139},
 'recall': {'mean': 0.972093, 'std': 0.009302},
 'EFO:EFO:0007033': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'HGNC:16507': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0.2, 'std': 0.4},
  'rc': {'mean': 0.2, 'std': 0.4}},
 'HGNC:6678': {'f1': {'mean': 0.920635, 'std': 0.06583},
  'pr': {'mean': 0.86, 'std': 0.115758},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:C034738': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'NCIT:NCIT:C60832': {'f1': {'mean': 0.981543, 'std': 0.006418},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.963833, 'std': 0.012344}}}

In [82]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [83]:
disamb.dump(model_name, results_path)

In [84]:
print(disamb.info())

Disambiguation model for SPO

Produces the disambiguations:
	LPO*	HGNC:6678
	Oxygen Saturation Measurement*	NCIT:NCIT:C60832
	SPO*	EFO:EFO:0007033
	SYNPR*	HGNC:16507
	sodium percarbonate	MESH:C034738

Class level metrics:
--------------------
Grounding                    	Count	F1     
Oxygen Saturation Measurement*	186	0.98154
                          LPO*	 22	0.92064
                        SYNPR*	  4	    0.2
                          SPO*	  3	    0.6
          sodium percarbonate	  1	    0.0

Global Metrics:
-----------------
	F1 score:	0.96985
	Precision:	0.96765
	Recall:		0.97209

* Positive labels
See Docstring for explanation



In [86]:
model_to_s3(disamb)

/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is 

In [26]:
from adeft.disambiguate import load_disambiguator

In [27]:
disamb = load_disambiguator("BAL")

In [28]:
disamb

In [29]:
print(_28.info())

Disambiguation model for BAL

Produces the disambiguations:
	Bronchoalveolar Lavage	MESH:D018893
	CEL*	HGNC:1848
	Liver, Artificial	MESH:D019164
	benzaldehyde lyase*	MESH:C059416
	betaine aldehyde*	CHEBI:CHEBI:15710
	dimercaprol*	CHEBI:CHEBI:64198

Class level metrics:
--------------------
Grounding             	Count	F1     
Bronchoalveolar Lavage	1259	 0.9929
                   CEL*	  36	    1.0
     Liver, Artificial	  18	0.83619
            Ungrounded	  17	   0.65
           dimercaprol*	   8	    0.4
    benzaldehyde lyase*	   3	    0.2
      betaine aldehyde*	   2	    0.2

Global Metrics:
-----------------
	F1 score:	0.90773
	Precision:	1.0
	Recall:		0.83293

* Positive labels
See Docstring for explanation



/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is 

In [85]:
p

dict_values(['In mammals, peroxidases have some organ, tissue, cellular and sub-cellular specific distribution patterns, performing some specific functions. These peroxidases include, glutathione peroxidase (GPx), myeloperoxidase (MPO), eosinophil peroxidase (EPO), uterine peroxidase, lactoperoxidase (LPO), salivary peroxidase (SPO) and thyroid peroxidase (TPO) (Klebanoff, 2005). These peroxidases play an important role in wide metabolic activities and are directly or indirectly involved in various diseases (Cheng et al., 2008).\nDifferent harmful microorganisms enter the human body through the oral passage regularly. These bacteria are killed by the first line of defence system present in saliva, which includes salivary peroxidase (SPO) as the major peroxidase. Oral peroxidases OPO are composed of salivary peroxidase (80%) and MPO (20%) (Pruitt et al., 1990; Nagler et al., 2002). SPO inhibits both Gram-positive and Gram-negative oral and non-oral bacteria. In addition to this, SPO sho